# Capacitated EOQ Model

## Model Inputs

Feel free to modify these to understand how the model works. Then `Run All`. 

In [ ]:
#This will be the start of the notebook

## Functions

In [ ]:
# All the functions we plan on using

## Model

In [ ]:
# Here's where the model runs

## Results

In [ ]:
# And here are the results!